In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

# Load data function

In [5]:
def load_data(filename):
    data = pd.read_csv(filename)
    X = data.drop('Revenue', axis=1) 
    y = data['Revenue']  

    return X, y

# Encoded features function

In [6]:
def encode_features(X):
    label_encoder = LabelEncoder()

    # Label encoding for 'Month'
    X['Month'] = label_encoder.fit_transform(X['Month'])

    # Label encoding for 'VisitorType'
    X['VisitorType'] = label_encoder.fit_transform(X['VisitorType'])

    return X

# Relu function

In [7]:
def relu(z):
    return np.maximum(0, z)

# Sigmoid function

In [8]:
def sigmoid(z):
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

# My dense vector function

In [9]:
def my_dense_v(A_in, W, b, g):
    Z = np.matmul(A_in, W) + b
    A_out = g(Z)
    return(A_out)

# My sequential function

In [10]:
def my_sequential_v(X, W1, b1, W2, b2, W3, b3):
    A1 = my_dense_v(X,  W1, b1, relu)
    A2 = my_dense_v(A1, W2, b2, relu)
    A3 = my_dense_v(A2, W3, b3, sigmoid)
    return(A3)

# Main 

In [11]:
# Load and preprocess data
X, y = load_data("dataset.csv")
X_encoded = encode_features(X)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train TensorFlow model
model = Sequential([
    tf.keras.Input(shape=(17,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
], name="my_model")

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(0.001))
model.fit(X_train_scaled, y_train, epochs=20)

# Extract weights after training
[layer1, layer2, layer3] = model.layers
W1_tmp, b1_tmp = layer1.get_weights()
W2_tmp, b2_tmp = layer2.get_weights()
W3_tmp, b3_tmp = layer3.get_weights()

# Use custom model for predictions on test data
custom_predictions = my_sequential_v(X_test_scaled, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp)
custom_predicted_classes = (custom_predictions > 0.5).astype(int)
custom_accuracy = accuracy_score(y_test, custom_predicted_classes)
print("Custom Model Accuracy:", custom_accuracy)

# TensorFlow model predictions on test data
tf_predictions = model.predict(X_test_scaled)
tf_predicted_classes = (tf_predictions > 0.5).astype(int)
tf_accuracy = accuracy_score(y_test, tf_predicted_classes)
print("TensorFlow Model Accuracy:", tf_accuracy)



Epoch 1/20

309/309 [==============================] - 1s 987us/step - loss: 0.3116
Epoch 2/20
309/309 [==============================] - 0s 971us/step - loss: 0.2612
Epoch 3/20
309/309 [==============================] - 0s 981us/step - loss: 0.2498
Epoch 4/20
309/309 [==============================] - 0s 961us/step - loss: 0.2439
Epoch 5/20
309/309 [==============================] - 0s 981us/step - loss: 0.2397
Epoch 6/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2354
Epoch 7/20
309/309 [==============================] - 0s 955us/step - loss: 0.2335
Epoch 8/20
309/309 [==============================] - 0s 955us/step - loss: 0.2307
Epoch 9/20
309/309 [==============================] - 0s 951us/step - loss: 0.2286
Epoch 10/20
309/309 [==============================] - 0s 964us/step - loss: 0.2251
Epoch 11/20
309/309 [==============================] - 0s 948us/step - loss: 0.2244
Epoch 12/20
309/309 [==============================] - 0s 981us/step - loss: 0.2210
E

Prediction = my_sequential_v(X, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
Prediction.shape

# Sensitivity, Specificity and F1 score

In [15]:
conf_matrix = confusion_matrix(tf_predicted_classes, y_test)

tn = conf_matrix[0, 0]  # True Negative
fp = conf_matrix[0, 1]  # False Positive
tp = conf_matrix[1, 1]  # True Positive
fn = conf_matrix[1, 0]  # False Negative

sensitivity = tp / (tp + fn)

specificity = tn/(tn+fp)

f1 = f1_score(tf_predicted_classes, y_test)

print("Sensitivity: {:.2f}".format(sensitivity))
print("Specificity: {:.2f}".format(specificity))
print("F1 Score: {:.2f}".format(f1))

Sensitivity: 0.72
Specificity: 0.91
F1 Score: 0.61
